## Load Dataset

In [1]:
!git clone https://github.com/declare-lab/MELD/

'MELD'에 복제합니다...
remote: Enumerating objects: 487, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 487 (delta 6), reused 0 (delta 0), pack-reused 475
오브젝트를 받는 중: 100% (487/487), 8.12 MiB | 22.78 MiB/s, 완료.
델타를 알아내는 중: 100% (254/254), 완료.


In [2]:
!pwd
!ls

/home/kwonyonggeun/workspace/DL/실무_자연어처리/실습
MELD  감정인식.ipynb


In [3]:
import glob
data_path = "./MELD/data/MELD/*.csv"
data_path_list = glob.glob(data_path)
print(data_path_list)

['./MELD/data/MELD/train_sent_emo.csv', './MELD/data/MELD/test_sent_emo.csv', './MELD/data/MELD/dev_sent_emo.csv']


## 데이터 확인

In [4]:
import csv
for data_path in data_path_list:
    f = open(data_path, 'r')
    rdr = csv.reader(f)

    for line in rdr:
        print(line)
        break

    f.close()
    break

['Sr No.', 'Utterance', 'Speaker', 'Emotion', 'Sentiment', 'Dialogue_ID', 'Utterance_ID', 'Season', 'Episode', 'StartTime', 'EndTime']


## 세션으로 데이터 분할하기

In [5]:
def split(session):
    final_data = []
    split_session = []
    
    for line in session:
        split_session.append(line)
        final_data.append(split_session[:])
    return final_data

for data_path in data_path_list:
    f = open(data_path, 'r')
    rdr = csv.reader(f)

    """ 세션 데이터 저장 """
    session_dataset = []
    session = []
    speaker_set = []

    """ 실제 데이터 저장 방식 """
    pre_sess = 'start'
    for i, line in enumerate(rdr):
        if i == 0:
            """ 저장할 데이터들 index 확인 """
            header = line
            utt_index = header.index('Utterance')
            speaker_index = header.index('Speaker')
            emo_index = header.index('Emotion')
            sess_index = header.index('Dialogue_ID')
        else:
            utt = line[utt_index]
            speaker = line[speaker_index]
            emotion = line[emo_index]
            sess = line[sess_index]
            
            """ 유니크한 Speaker로 바꾸기 """
            if speaker in speaker_set: # 이미 등장한 Speaker라면
                uniq_speaker = speaker_set.index(speaker)            
            else: # 등장하지 않은 Speaker 
                speaker_set.append(speaker)
                uniq_speaker = speaker_set.index(speaker)

            if pre_sess == 'start' or sess == pre_sess: # 시작이면 바로 저장 또는 
                session.append([uniq_speaker, utt, emotion])
            else:
                """ 세션 데이터 저장 """
                session_dataset += split(session)
                session = [[uniq_speaker, utt, emotion]]
                speaker_set = []
            pre_sess = sess

    """ 마지막 세션 저장 """
    session_dataset += split(session)
    f.close()

    """ 데이터 분할하기 """
    break

In [6]:
# 첫번째 발화에 대해서 sadness라고 학습시키기 위한 data
session_dataset[0]

[[0,
  'also I was the point person on my company’s transition from the KL-5 to GR-6 system.',
  'neutral']]

In [7]:
# 6번째 발화가 neutral이라고 학습시키기 위한 data
session_dataset[5]

[[0,
  'also I was the point person on my company’s transition from the KL-5 to GR-6 system.',
  'neutral'],
 [1, 'You must’ve had your hands full.', 'neutral'],
 [0, 'That I did. That I did.', 'neutral'],
 [1, 'So let’s talk a little bit about your duties.', 'neutral'],
 [0, 'My duties?  All right.', 'surprise'],
 [1,
  'Now you’ll be heading a whole division, so you’ll have a lot of duties.',
  'neutral']]

## 배치처리 알아보기

In [11]:
import csv
from torch.utils.data import Dataset
def split(session):
    final_data = []
    split_session = []
    for line in session:
        split_session.append(line)
        final_data.append(split_session[:])
    return final_data

class data_loader(Dataset):
    def __init__(self, data_path):
        f = open(data_path, 'r')
        rdr = csv.reader(f)

        """ 추가 """
        emoSet = set()

        self.session_dataset = []
        session = []
        speaker_set = []

        """ 실제 데이터 저장 방식 """
        pre_sess = 'start'
        for i, line in enumerate(rdr):
            if i == 0:
                """ 저장할 데이터들 index 확인 """
                header = line
                utt_index = header.index('Utterance')
                speaker_index = header.index('Speaker')
                emo_index = header.index('Emotion')
                sess_index = header.index('Dialogue_ID')
            else:
                utt = line[utt_index]
                speaker = line[speaker_index]
                emotion = line[emo_index]
                sess = line[sess_index]
                
                """ 유니크한 Speaker로 바꾸기 """
                if speaker in speaker_set: # 이미 등장한 Speaker라면
                    uniq_speaker = speaker_set.index(speaker)            
                else: # 등장하지 않은 Speaker 
                    speaker_set.append(speaker)
                    uniq_speaker = speaker_set.index(speaker)

                if pre_sess == 'start' or sess == pre_sess: # 시작이면 바로 저장 또는 
                    session.append([uniq_speaker, utt, emotion])
                else:
                    """ 세션 데이터 저장 """
                    self.session_dataset += split(session)
                    session = [[uniq_speaker, utt, emotion]]
                    speaker_set = []
                pre_sess = sess
        """ 마지막 세션 저장 """
        self.session_dataset += split(session)
        f.close()

        """ 추가 """
        self.emoList = sorted(emoSet) # 항상 같은 레이블 순서를 유지하기 위해
        f.close()

    def __len__(self):
        return len(self.session_dataset)
    
    def __getitem__(self, idx):
        return self.session_dataset[idx]
    
    def collate_fn(self, sessions): # 배치를 위한 구성
        '''
            input:
                data: [(session1), (session2), ...]
            return:
                batch_input_tokens_pad: (B, L) padded
                batch_labels: (B)
        '''
        batch_input_token = []
        for session in sessions:
            input_token = ""
            for line in session:
                speaker, utt, emotion = line
                input_token += utt
            batch_input_token.append(input_token)

        return batch_input_token
            
    

In [17]:
dev_dataset = data_loader('./MELD/data/MELD/dev_sent_emo.csv')
dev_dataset[0], dev_dataset[1]

([[0, 'Oh my God, he’s lost it. He’s totally lost it.', 'sadness']],
 [[0, 'Oh my God, he’s lost it. He’s totally lost it.', 'sadness'],
  [1, 'What?', 'surprise']])

In [23]:
# 배치 결과 확인
from torch.utils.data import DataLoader
dev_dataloader = DataLoader(dev_dataset, batch_size=2, shuffle=False, num_workers=4, collate_fn=dev_dataset.collate_fn)

i = 0
for data in dev_dataloader:
    print(i, data)
    i += 1
    if i > 2:
        break

0 ['Oh my God, he’s lost it. He’s totally lost it.', 'Oh my God, he’s lost it. He’s totally lost it.What?']
1 ['Or! Or, we could go to the bank, close our accounts and cut them off at the source.', 'Or! Or, we could go to the bank, close our accounts and cut them off at the source.You’re a genius!']
2 ['Or! Or, we could go to the bank, close our accounts and cut them off at the source.You’re a genius!Aww, man, now we won’t be bank buddies!', 'Or! Or, we could go to the bank, close our accounts and cut them off at the source.You’re a genius!Aww, man, now we won’t be bank buddies!Now, there’s two reasons.']


In [25]:
# 배치 결과 확인
from torch.utils.data import DataLoader
dev_dataloader = DataLoader(dev_dataset, batch_size=3, shuffle=False, num_workers=4, collate_fn=dev_dataset.collate_fn)

for data in dev_dataloader:
    print(data[0])
    print(data[1])
    print(data[2])
    break

Oh my God, he’s lost it. He’s totally lost it.
Oh my God, he’s lost it. He’s totally lost it.What?
Or! Or, we could go to the bank, close our accounts and cut them off at the source.


## 사전 학습 모델들에 대한 백그라운드

In [2]:
from transformers import RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [3]:
dir(tokenizer)

['SPECIAL_TOKENS_ATTRIBUTES',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_tokens',
 '_additional_special_tokens',
 '_batch_encode_plus',
 '_batch_prepare_for_model',
 '_bos_token',
 '_build_conversation_input_ids',
 '_cls_token',
 '_convert_id_to_token',
 '_convert_token_to_id',
 '_convert_token_to_id_with_added_voc',
 '_decode',
 '_decode_use_source_tokenizer',
 '_encode_plus',
 '_eos_token',
 '_eventual_warn_about_too_long_sequence',
 '_from_pretrained',
 '_get_padding_truncation_strategies',
 '_mask_token',
 '_pad',
 '_pad_token',
 '_pad_token_type_id',
 '_save_pretrained',
 '_sep_token',
 '_tokenize',
 '_unk_to

In [4]:
print(tokenizer.cls_token, tokenizer.sep_token, tokenizer.pad_token)

<s> </s> <pad>


In [5]:
tokenizer.model_max_length

512

In [10]:
""" 토크나이저 작동 """
res = tokenizer('hello, this is fastcampus')
print(res) # 0과 2가 special token
res = tokenizer.encode('hello, this is fastcampus')
print(res) # 토큰의 결과만 뽑아냄
res = tokenizer(['hello, this is fastcampus', 'what are you doing?'])
print(res)
res = tokenizer(['hello, this is fastcampus', 'what are you doing?'], add_special_tokens=False)
print(res)

{'input_ids': [0, 42891, 6, 42, 16, 1769, 28135, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}
[0, 42891, 6, 42, 16, 1769, 28135, 2]
{'input_ids': [[0, 42891, 6, 42, 16, 1769, 28135, 2], [0, 12196, 32, 47, 608, 116, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1]]}
{'input_ids': [[42891, 6, 42, 16, 1769, 28135], [12196, 32, 47, 608, 116]], 'attention_mask': [[1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]}


In [22]:
from dataset import data_loader
from torch.utils.data import DataLoader
dev_dataset = data_loader('./MELD/data/MELD/dev_sent_emo.csv')
dev_dataloader = DataLoader(dev_dataset, batch_size=3, shuffle=False, num_workers=4, collate_fn=dev_dataset.collate_fn)

for i, data in enumerate(dev_dataloader):
    if i == 1:
        print(data[0].shape)
        batch_padding_token, batch_padding_attention_mask, batch_PM_input, batch_label = data
        print("batch_padding_token : ", batch_padding_token)
        print("batch_padding_attention_mask : ", batch_padding_attention_mask)
        print("batch_PM_input : ", batch_PM_input)
        print("batch_label : ", batch_label)
        break

torch.Size([3, 58])
batch_padding_token :  tensor([[    0,  1793,   328,  1793,     6,    52,   115,   213,     7,     5,
           827,     6,   593,    84,  2349,     8,   847,   106,   160,    23,
             5,  1300,     4,     2,   370,    17,    27,   241,    10, 16333,
           328,     2,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1],
        [    0,  1793,   328,  1793,     6,    52,   115,   213,     7,     5,
           827,     6,   593,    84,  2349,     8,   847,   106,   160,    23,
             5,  1300,     4,     2,   370,    17,    27,   241,    10, 16333,
           328,     2,    83, 33130,     6,   313,     6,   122,    52,   351,
            17,    27,    90,    28,   827, 30489,   328,     2,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1],
        [    0,  1793,   328,  1793

## 사전학습 모델 불러와서 로딩하기

In [23]:
from transformers import RobertaModel
model = RobertaModel.from_pretrained('roberta-base')

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

In [24]:
model

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropou

## 사전학습 모델 사용해보기

In [27]:
# batch가 3이고, 총 length가 58인 상태의 입력이 들어감
batch_padding_token.shape, batch_padding_attention_mask.shape

(torch.Size([3, 58]), torch.Size([3, 58]))

In [30]:
# 입력의 길이가 맞지 않는 경우에는 padding(pad token = 1)처리해줬는데, 이 경우에는 의미가 없으므로 attention mask가 0으로 되어 있음
batch_padding_token, batch_padding_attention_mask

(tensor([[    0,  1793,   328,  1793,     6,    52,   115,   213,     7,     5,
            827,     6,   593,    84,  2349,     8,   847,   106,   160,    23,
              5,  1300,     4,     2,   370,    17,    27,   241,    10, 16333,
            328,     2,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1],
         [    0,  1793,   328,  1793,     6,    52,   115,   213,     7,     5,
            827,     6,   593,    84,  2349,     8,   847,   106,   160,    23,
              5,  1300,     4,     2,   370,    17,    27,   241,    10, 16333,
            328,     2,    83, 33130,     6,   313,     6,   122,    52,   351,
             17,    27,    90,    28,   827, 30489,   328,     2,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1],
         [    0,  1793,   328,  1793,     6,    52,   115,   213, 

In [32]:
batch_com_out = model(input_ids = batch_padding_token, attention_mask = batch_padding_attention_mask)['last_hidden_state']
print(batch_com_out.shape)
batch_com_final = batch_com_out[:,0,:] # CLS 토큰의 output을 가져오기 위해
print(batch_com_final.shape)

torch.Size([3, 58, 768])
torch.Size([3, 768])


In [33]:
result = model(input_ids = batch_padding_token, attention_mask = batch_padding_attention_mask)
result.keys()

odict_keys(['last_hidden_state', 'pooler_output'])